In [11]:
import pandas as pd

from fairfl.model.preprocess import ACSIncomePreprocess
from fairfl.model.model import ACSIncomeModel
from fairfl.metrics.bias_metrics import BiasType
from fairfl.utils.read_data import DatasetUtils

In [12]:
bias_type = BiasType.race
states_2 = ["MT", "NH", "ND", "ME", "SD"]
preprocess = ACSIncomePreprocess()
model = ACSIncomeModel(bias_type=bias_type)
utils = DatasetUtils()

In [13]:
def get_states(train, states):
    train_states = utils.get_selected_states(train, states)
    return train_states

In [4]:
train, test = preprocess.get_preprocessed_data()

In [14]:
train_states = get_states(train, states_2)

In [15]:
train_states

,AGEP,SCHL,POBP,WKHP,PINCP,COW_1.0,COW_2.0,COW_3.0,COW_4.0,COW_5.0,...,RELP_12.0,RELP_13.0,RELP_14.0,RELP_15.0,RELP_16.0,RELP_17.0,RAC1P_Black,RAC1P_others,SEX_Female,SEX_Male
56,0.0,6.0,26.0,1.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
94,2.0,6.0,23.0,2.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
142,1.0,1.0,50.0,1.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
161,1.0,4.0,34.0,1.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
308,1.0,7.0,211.0,1.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164869,0.0,1.0,11.0,1.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1165014,3.0,1.0,46.0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1165081,2.0,7.0,23.0,1.0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,1
1165137,1.0,6.0,6.0,2.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [16]:
test2 = test.drop('ST', axis=1)
_ = model.print_model_metrics(train_states, test2)

/home/safir/.virtualenvs/ACIncome_bias_fl/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
import pickle

l_model = pickle.load(open("lr_selected.pkl", "rb"))

In [18]:
states_result_dfs = list()

for state in states_2:
    res = model.print_loaded_model_metrics(test, l_model, state)
    states_result_dfs.append(res)

In [19]:
pd.concat(states_result_dfs).to_excel(f"{bias_type}_c_new3.xlsx", index=False)